# Database Execution Notebook MC536 Project

This notebook is divided into sections for importing requirements, creating the database and tables, and adding data to the tables.

## 1. Requirements

In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import os
from datetime import datetime
import numpy as np
import unicodedata

# Database configuration parameters
DB_CONFIG = {
    'dbname': 'my_database',  # Replace with your database name
    'user': 'postgres',         # Replace with your username
    'password': 'mypassword',     # Replace with your password
    'host': '127.0.0.1',
    'port': '5433'
}

# Path configuration for csv files
DATA_DIR = './data'

# Path to the SQL script for creating the database
DB_script_path = './DB_creation_script.sql'

## 2. Connecting to Database Function

In [2]:
# Function to connect to PostgreSQL - updated to set encoding
def connect_to_db():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        conn.set_client_encoding('UTF8')  # Set connection encoding to UTF-8
        print("Connected to PostgreSQL database successfully")
        return conn
    except (Exception, psycopg2.Error) as error:
        print(f"Error while connecting to PostgreSQL: {error}")
        return None

## 3. Data Loading and Inserting Functions

In [3]:
# Function to normalize accented text
def normalize_text(text):
    if not isinstance(text, str):
        return text
    # Normalize accented characters properly
    return text.strip()

# Function to load CAPACIDADE_GERACAO.csv
def load_capacidade_geracao():
    file_path = os.path.join(DATA_DIR, 'CAPACIDADE_GERACAO.csv')
    try:
        # Try multiple encodings to find the correct one
        encodings = ['utf-8', 'latin-1', 'ISO-8859-1', 'cp1252']
        
        for encoding in encodings:
            try:
                print(f"Attempting to read with encoding: {encoding}")
                # Read CSV with the specified encoding
                df = pd.read_csv(file_path, delimiter=';', encoding=encoding)
                
                # Replace empty strings and 'NULL' with None
                df = df.replace(['', 'NULL'], None)
                
                # Normalize text in all string columns to handle accents properly
                for col in df.select_dtypes(include=['object']).columns:
                    df[col] = df[col].apply(normalize_text)
                
                # Convert date columns to datetime
                for date_col in ['dat_entradateste', 'dat_entradaoperacao', 'dat_desativacao']:
                    if date_col in df.columns:
                        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
                
                # Convert potencia_efetiva to float
                if 'val_potenciaefetiva' in df.columns:
                    df['val_potenciaefetiva'] = pd.to_numeric(df['val_potenciaefetiva'], errors='coerce')
                
                print(f"Successfully loaded data with encoding: {encoding}")
                return df
                
            except UnicodeDecodeError:
                print(f"Failed to decode with {encoding}, trying next encoding...")
        
        raise Exception("Could not read the file with any of the attempted encodings")
    except Exception as e:
        print(f"Error loading CAPACIDADE_GERACAO.csv: {e}")
        return None
    
# Function to load country data files
def load_country_data(file_name, value_column_name):
    file_path = os.path.join(DATA_DIR, file_name)
    try:
        df = pd.read_csv(file_path)
        
        # Extract relevant columns
        df = df[['Entity', 'Code', 'Year', value_column_name]]
        
        # Rename columns for consistency
        df = df.rename(columns={
            'Entity': 'nome_pais',
            'Code': 'code',
            'Year': 'ano',
            value_column_name: 'valor'
        })
        
        # Convert numeric values
        df['ano'] = pd.to_numeric(df['ano'], errors='coerce')
        df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
        
        # Remove rows with NaN in code field
        df = df.dropna(subset=['code'])
        
        return df
    except Exception as e:
        print(f"Error loading {file_name}: {e}")
        return None
    
# Function to insert regiao data
def insert_subsistema(conn, df):
    try:
        cursor = conn.cursor()
        
        # Get the country id for Brazil
        cursor.execute("SELECT id_pais FROM Pais WHERE nome LIKE '%Brazil%' OR nome LIKE '%Brasil%' LIMIT 1")
        brasil_id = cursor.fetchone()
        
        if not brasil_id:
            # If Brazil doesn't exist, insert it
            cursor.execute(
                """
                INSERT INTO Pais (code, nome)
                VALUES (%s, %s)
                RETURNING id_pais
                """,
                ('BRA', 'Brazil')
            )
            brasil_id = cursor.fetchone()
        
        # Extract unique regions from the dataset using id_subsistema and nom_subsistema
        subsistemas_df = df[['id_subsistema', 'nom_subsistema']].drop_duplicates()
        subsistemas_ids = {}
        
        # Insert each unique region
        for _, row in subsistemas_df.iterrows():
            cod_subsistema = row['id_subsistema']  # Changed from cod_regiao to cod_subsistema
            nome = row['nom_subsistema']
            
            # First check if region already exists
            cursor.execute(
                """
                SELECT id_subsistema FROM Subsistema WHERE cod_subsistema = %s
                """,
                (cod_subsistema,)
            )
            existing = cursor.fetchone()
            
            if existing:
                # Region exists, update if needed
                cursor.execute(
                    """
                    UPDATE Subsistema SET nome = %s WHERE cod_subsistema = %s
                    RETURNING id_subsistema
                    """,
                    (nome, cod_subsistema)
                )
                subsistema_id = cursor.fetchone()[0]
                subsistemas_ids[nome] = subsistema_id
            else:
                # Insert new region
                cursor.execute(
                    """
                    INSERT INTO Subsistema (cod_subsistema, nome, id_pais)
                    VALUES (%s, %s, %s)
                    RETURNING id_subsistema
                    """,
                    (cod_subsistema, nome, brasil_id[0])
                )
                subsistema_id = cursor.fetchone()[0]
                subsistemas_ids[cod_subsistema] = subsistema_id
        
        conn.commit()
        print(f"Inserted/updated {len(subsistemas_ids)} subsistemas")
        
        return subsistemas_ids
    except Exception as e:
        print(f"Error inserting into Subsistema: {e}")
        conn.rollback()
        return {}

# Function to insert estado and subsistema_estado data
def insert_estado_and_subsistema_estado(conn, df):
    try:
        cursor = conn.cursor()
        
        # Extract unique states from the dataset
        estados_df = df[['id_estado', 'nom_estado']].drop_duplicates()
        estados_ids = {}
        
        # Insert each unique state
        for _, row in estados_df.iterrows():
            cod_estado = row['id_estado']  # Use id_estado from the DataFrame as cod_estado in the database
            nome_estado = row['nom_estado']
            
            # First check if the state already exists
            cursor.execute(
                """
                SELECT id_estado FROM Estado WHERE cod_estado = %s
                """,
                (cod_estado,)
            )
            existing = cursor.fetchone()
            
            if existing:
                # State exists, update if needed
                cursor.execute(
                    """
                    UPDATE Estado SET nome = %s WHERE cod_estado = %s
                    RETURNING id_estado
                    """,
                    (nome_estado, cod_estado)
                )
                estado_id = cursor.fetchone()[0]
                estados_ids[cod_estado] = estado_id
            else:
                # Insert new state
                cursor.execute(
                    """
                    INSERT INTO Estado (cod_estado, nome)
                    VALUES (%s, %s)
                    RETURNING id_estado
                    """,
                    (cod_estado, nome_estado)
                )
                estado_id = cursor.fetchone()[0]
                estados_ids[cod_estado] = estado_id
        
        conn.commit()
        print(f"Inserted/updated {len(estados_ids)} states")
        
        # Insert Subsistemas
        subsistemas_ids = insert_subsistema(conn, df)
        
        # Insert into Subsistema_Estado table
        subsistema_estado_df = df[['id_estado', 'id_subsistema']].drop_duplicates()
        subsistema_estado_ids = {}
        
        for _, row in subsistema_estado_df.iterrows():
            cod_estado = row['id_estado']  # Use id_estado from the DataFrame as cod_estado in the database
            cod_subsistema = row['id_subsistema'] # Use id_subsistema from the DataFrame as cod_subsistema in the database
            
            # Get the artificial primary key for the state
            estado_id = estados_ids.get(cod_estado)
            subsistema_id = subsistemas_ids.get(cod_subsistema)
            
            
            if estado_id:
                # Check if the relationship already exists
                cursor.execute(
                    """
                    SELECT id_estado, id_subsistema FROM Subsistema_Estado
                    WHERE id_estado = %s AND id_subsistema = %s
                    """,
                    (estado_id, subsistema_id)
                )
                existing = cursor.fetchone()
                
                if not existing:
                    # Insert new relationship
                    cursor.execute(
                        """
                        INSERT INTO Subsistema_Estado (id_estado, id_subsistema)
                        VALUES (%s, %s)
                        RETURNING id_estado, id_subsistema
                        """,
                        (estado_id, subsistema_id)
                    )
                    subsistema_estado_ids[(estado_id, subsistema_id)] = cursor.fetchone()
        
        conn.commit()
        print(f"Inserted/updated {len(subsistema_estado_ids)} subsistema-estado relationships")
        
    except Exception as e:
        print(f"Error inserting into Estado and Subsistema_Estado: {e}")
        conn.rollback()

# Function to insert data into Agente_Proprietario table
def insert_agentes(conn, df):
    try:
        cursor = conn.cursor()
        
        # Extract unique agents from the dataframe
        agentes = df['nom_agenteproprietario'].dropna().unique()
        
        for agente in agentes:
            cursor.execute(
                """
                INSERT INTO Agente_Proprietario (nome)
                VALUES (%s)
                ON CONFLICT DO NOTHING
                """,
                (agente,)
            )
            
        conn.commit()
        print(f"Inserted {len(agentes)} agents")
    except Exception as e:
        print(f"Error inserting into Agente_Proprietario: {e}")
        conn.rollback()
        
# Function to insert data into Usina table
def insert_usinas(conn, df):
    try:
        cursor = conn.cursor()
        inserted = 0
        
        # First, get mappings for foreign keys
        cursor.execute("SELECT id_estado, cod_estado FROM Estado")
        estado_map = {cod: id for id, cod in cursor.fetchall()}
        
        cursor.execute("SELECT id_agente, nome FROM Agente_Proprietario")
        agente_map = {nome: id for id, nome in cursor.fetchall()}
        
        # Group by unique usina - now including the ceg field
        usinas = df[['nom_usina', 'nom_agenteproprietario', 'nom_tipousina',
                     'nom_modalidadeoperacao', 'id_estado', 'ceg']].drop_duplicates()
        
        for _, row in usinas.iterrows():
            id_agente = agente_map.get(row['nom_agenteproprietario'])
            id_estado = estado_map.get(row['id_estado'])
            
            # Ensure proper handling of accented characters
            tipo_usina = row['nom_tipousina'].strip() if isinstance(row['nom_tipousina'], str) else row['nom_tipousina']
            modalidade = row['nom_modalidadeoperacao'].strip() if isinstance(row['nom_modalidadeoperacao'], str) else row['nom_modalidadeoperacao']
            nome_usina = row['nom_usina'].strip() if isinstance(row['nom_usina'], str) else row['nom_usina']
            ceg = row['ceg'].strip() if isinstance(row['ceg'], str) else row['ceg']
            
            cursor.execute(
                """
                INSERT INTO Usina (nome, id_agente_proprietario, tipo,
                                  modalidade_operacao, id_estado, ceg)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT DO NOTHING
                RETURNING id_usina
                """,
                (nome_usina, id_agente, tipo_usina, modalidade, id_estado, ceg)
            )
            
            result = cursor.fetchone()
            if result:
                inserted += 1
                
        conn.commit()
        print(f"Inserted {inserted} power plants")
    except Exception as e:
        print(f"Error inserting into Usina: {e}")
        conn.rollback()
        
# Function to insert data into Unidade_Geradora table
def insert_unidades_geradoras(conn, df):
    try:
        cursor = conn.cursor()
        inserted = 0
        
        # Get usina mapping
        cursor.execute("SELECT id_usina, nome FROM Usina")
        usina_map = {nome: id for id, nome in cursor.fetchall()}
        
        # Extract units - now including date fields that were moved from Usina
        unidades = df[['nom_usina', 'cod_equipamento', 'nom_unidadegeradora',
                      'num_unidadegeradora', 'dat_entradateste', 'dat_entradaoperacao',
                      'dat_desativacao', 'val_potenciaefetiva', 
                      'nom_combustivel']].drop_duplicates()
        
        for _, row in unidades.iterrows():
            id_usina = usina_map.get(row['nom_usina'])
            
            # Convert num_unidadegeradora to integer safely
            try:
                num_unidade = int(row['num_unidadegeradora'])
            except (ValueError, TypeError):
                # If conversion fails, set to None
                num_unidade = None
            
            # Handle NaT/None values for date columns
            data_teste = None if pd.isna(row['dat_entradateste']) else row['dat_entradateste']
            data_operacao = None if pd.isna(row['dat_entradaoperacao']) else row['dat_entradaoperacao']
            data_desativacao = None if pd.isna(row['dat_desativacao']) else row['dat_desativacao']
            
            # Handle accented characters properly
            combustivel = row['nom_combustivel'].strip() if isinstance(row['nom_combustivel'], str) else row['nom_combustivel']
            nome_unidade = row['nom_unidadegeradora'].strip() if isinstance(row['nom_unidadegeradora'], str) else row['nom_unidadegeradora']
            cod_equip = row['cod_equipamento'].strip() if isinstance(row['cod_equipamento'], str) else row['cod_equipamento']
            
            if id_usina:
                cursor.execute(
                    """
                    INSERT INTO Unidade_Geradora (cod_equipamento, nome_unidade,
                                               num_unidade, data_entrada_teste,
                                               data_entrada_operacao, data_desativacao,
                                               potencia_efetiva, combustivel, id_usina)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT DO NOTHING
                    RETURNING id_unidade
                    """,
                    (cod_equip, nome_unidade,
                     num_unidade, data_teste, data_operacao,
                     data_desativacao, row['val_potenciaefetiva'],
                     combustivel, id_usina)
                )
                
                result = cursor.fetchone()
                if result:
                    inserted += 1
                    
        conn.commit()
        print(f"Inserted {inserted} generating units")
    except Exception as e:
        print(f"Error inserting into Unidade_Geradora: {e}")
        conn.rollback()
        
# Function to insert data into Pais table
def insert_paises(conn, df):
    try:
        cursor = conn.cursor()
        
        # Extract unique countries and ensure code is not NaN
        paises = df[['nome_pais', 'code']].dropna(subset=['code']).drop_duplicates()
        
        inserted = 0
        country_ids = {}  # Dictionary to store code -> id_pais mapping
        
        for _, row in paises.iterrows():
            # First check if country exists
            cursor.execute(
                """
                SELECT id_pais FROM Pais WHERE code = %s
                """,
                (row['code'],)
            )
            existing = cursor.fetchone()
            
            if existing:
                country_ids[row['code']] = existing[0]
            else:
                cursor.execute(
                    """
                    INSERT INTO Pais (code, nome)
                    VALUES (%s, %s)
                    RETURNING id_pais
                    """,
                    (row['code'], row['nome_pais'])
                )
                
                result = cursor.fetchone()
                if result:
                    inserted += 1
                    country_ids[row['code']] = result[0]
                
        conn.commit()
        print(f"Inserted {inserted} countries")
        return country_ids
    except Exception as e:
        print(f"Error inserting into Pais: {e}")
        conn.rollback()
        return {}
    
# Function to insert data into a specified table
def insert_country_data(conn, df, table_name):
    try:
        cursor = conn.cursor()
        inserted = 0
        
        # Filter out rows with NaN codes
        df_clean = df.dropna(subset=['code'])
        
        # Get country_id mapping
        country_ids = {}
        cursor.execute("SELECT id_pais, code FROM Pais")
        for id_pais, code in cursor.fetchall():
            country_ids[code] = id_pais
        
        for _, row in df_clean.iterrows():
            code = row['code']
            id_pais = country_ids.get(code)
            
            if id_pais:
                cursor.execute(
                    f"""
                    INSERT INTO {table_name} (id_pais, ano, porcentagem)
                    VALUES (%s, %s, %s)
                    ON CONFLICT DO NOTHING
                    RETURNING id
                    """,
                    (id_pais, row['ano'], row['valor'])
                )
                
                result = cursor.fetchone()
                if result:
                    inserted += 1
            else:
                print(f"Warning: No country ID found for code {code}")
                
        conn.commit()
        print(f"Inserted {inserted} records into {table_name}")
    except Exception as e:
        print(f"Error inserting into {table_name}: {e}")
        conn.rollback()
        

# Function to insert investment data
def insert_investment_data(conn, df):
    try:
        cursor = conn.cursor()
        inserted = 0
        
        # Filter out rows with NaN codes
        df_clean = df.dropna(subset=['code'])
        
        # Get country_id mapping
        country_ids = {}
        cursor.execute("SELECT id_pais, code FROM Pais")
        for id_pais, code in cursor.fetchall():
            country_ids[code] = id_pais
            
        for _, row in df_clean.iterrows():
            code = row['code']
            id_pais = country_ids.get(code)
            
            if id_pais:
                cursor.execute(
                    """
                    INSERT INTO Investimento_Energia_Limpa (id_pais, ano, valor_dolar)
                    VALUES (%s, %s, %s)
                    ON CONFLICT DO NOTHING
                    RETURNING id
                    """,
                    (id_pais, row['ano'], row['valor'])
                )
                
                result = cursor.fetchone()
                if result:
                    inserted += 1
            else:
                print(f"Warning: No country ID found for code {code}")
                
        conn.commit()
        print(f"Inserted {inserted} investment records")
    except Exception as e:
        print(f"Error inserting into Investimento_Energia_Limpa: {e}")
        conn.rollback()
        
        
# Function to insert renewable per capita data
def insert_renewable_per_capita(conn, df):
    try:
        cursor = conn.cursor()
        inserted = 0
        
        # Filter out rows with NaN codes
        df_clean = df.dropna(subset=['code'])
        
        # Get country_id mapping
        country_ids = {}
        cursor.execute("SELECT id_pais, code FROM Pais")
        for id_pais, code in cursor.fetchall():
            country_ids[code] = id_pais
        
        for _, row in df_clean.iterrows():
            code = row['code']
            id_pais = country_ids.get(code)
            
            if id_pais:
                cursor.execute(
                    """
                    INSERT INTO Energia_Renovavel_Per_Capita (id_pais, ano, geracao_watts)
                    VALUES (%s, %s, %s)
                    ON CONFLICT DO NOTHING
                    RETURNING id
                    """,
                    (id_pais, row['ano'], row['valor'])
                )
                
                result = cursor.fetchone()
                if result:
                    inserted += 1
            else:
                print(f"Warning: No country ID found for code {code}")
                
        conn.commit()
        print(f"Inserted {inserted} renewable per capita records")
    except Exception as e:
        print(f"Error inserting into Energia_Renovavel_Per_Capita: {e}")
        conn.rollback()
        
        
# Function to load and process HDI data
def load_hdi_data():
    file_path = os.path.join(DATA_DIR, 'HDR23-24_Composite_indices_complete_time_series.csv')
    
    # List of encodings to try
    encodings = ['latin-1', 'ISO-8859-1', 'cp1252', 'utf-8-sig']
    
    for encoding in encodings:
        try:
            print(f"Trying to read HDI data with encoding: {encoding}")
            # Read CSV file with the current encoding
            df = pd.read_csv(file_path, encoding=encoding)
            
            # Get the column names that contain HDI data (columns starting with 'hdi_')
            hdi_columns = [col for col in df.columns if col.startswith('hdi_') and col != 'hdicode']
            
            # Create a list to store the data in the format (country_code, year, value)
            hdi_data = []
            
            # Process each country
            for _, row in df.iterrows():
                iso3 = row['iso3']
                country_name = row['country']
                
                # Process each year's HDI value
                for col in hdi_columns:
                    # Extract the year from the column name (format: hdi_YYYY)
                    year = col.split('_')[1]
                    
                    # Try to convert to integer (handle cases like 'hdi_rank' if they exist)
                    try:
                        year = int(year)
                    except ValueError:
                        continue
                    
                    # Get the HDI value
                    hdi_value = row[col]
                    
                    # Only add if the HDI value is not null
                    if pd.notna(hdi_value):
                        hdi_data.append({
                            'code': iso3,
                            'nome_pais': country_name,
                            'ano': year,
                            'valor': float(hdi_value)
                        })
            
            # Convert list to DataFrame
            hdi_df = pd.DataFrame(hdi_data)
            print(f"Successfully loaded HDI data with encoding: {encoding}")
            return hdi_df
        
        except Exception as e:
            print(f"Failed with encoding {encoding}: {e}")
            continue
    
    # If all encodings fail
    print("Error: Could not load HDI data with any of the attempted encodings")
    return None


# Function to insert HDI data
def insert_hdi_data(conn, df):
    try:
        cursor = conn.cursor()
        inserted = 0
        
        # Filter out rows with NaN codes
        df_clean = df.dropna(subset=['code'])
        
        # Get country_id mapping
        country_ids = {}
        cursor.execute("SELECT id_pais, code FROM Pais")
        for id_pais, code in cursor.fetchall():
            country_ids[code] = id_pais
        
        # Add any missing countries
        new_countries = []
        for _, row in df_clean.iterrows():
            if row['code'] not in country_ids:
                new_countries.append((row['code'], row['nome_pais']))
        
        # Insert new countries if any
        if new_countries:
            new_countries_unique = list(set(new_countries))
            for code, nome in new_countries_unique:
                cursor.execute(
                    """
                    INSERT INTO Pais (code, nome)
                    VALUES (%s, %s)
                    RETURNING id_pais
                    """,
                    (code, nome)
                )
                result = cursor.fetchone()
                if result:
                    country_ids[code] = result[0]
        
        # Now insert HDI data
        for _, row in df_clean.iterrows():
            code = row['code']
            id_pais = country_ids.get(code)
            
            if id_pais:
                cursor.execute(
                    """
                    INSERT INTO IDH (id_pais, ano, indice)
                    VALUES (%s, %s, %s)
                    ON CONFLICT DO NOTHING
                    RETURNING id
                    """,
                    (id_pais, row['ano'], row['valor'])
                )
                
                result = cursor.fetchone()
                if result:
                    inserted += 1
            else:
                print(f"Warning: No country ID found for HDI code {code}")
                
        conn.commit()
        print(f"Inserted {inserted} HDI records")
    except Exception as e:
        print(f"Error inserting into IDH: {e}")
        conn.rollback()

## 4. Database Server Creation

In [4]:
# Connect to the PostgreSQL database
conn = connect_to_db()
if conn:
    print("Database connection established.")
else:
    print("Failed to connect to the database.")

Error while connecting to PostgreSQL: connection to server at "127.0.0.1", port 5433 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

Failed to connect to the database.


## 5. Create Tables

In [5]:
# Read and execute the SQL script to create tables
sql_file_path = DB_script_path

try:
    with open(sql_file_path, 'r') as file:
        sql_script = file.read()
    
    cursor = conn.cursor()
    cursor.execute(sql_script)
    conn.commit()
    print("Tables created successfully.")
except Exception as e:
    print(f"Error creating tables: {e}")
finally:
    cursor.close()

Error creating tables: 'NoneType' object has no attribute 'cursor'


NameError: name 'cursor' is not defined

## 5. Add Data to Tables

### 5.1 Load and Insert `CAPACIDADE_GERACAO.csv`

In [ ]:
# Load and insert data from CAPACIDADE_GERACAO.csv
cap_data = load_capacidade_geracao()
if cap_data is not None:
    insert_estado_and_subsistema_estado(conn, cap_data)
    insert_agentes(conn, cap_data)
    insert_usinas(conn, cap_data)
    insert_unidades_geradoras(conn, cap_data)
else:
    print("Failed to load CAPACIDADE_GERACAO.csv.")

Attempting to read with encoding: utf-8
Successfully loaded data with encoding: utf-8
Inserted/updated 28 states
Inserted/updated 5 subsistemas
Inserted/updated 30 subsistema-estado relationships
Inserted 1441 agents
Inserted 1896 power plants
Inserted 5189 generating units


### 5.2 Load and Insert Country Data

In [ ]:
# Load and insert country data
country_files = [
    ('share-of-the-population-with-access-to-electricity.csv', 'Access to electricity (% of population)', 'Acesso_Eletricidade'),
    ('access-to-clean-fuels-and-technologies-for-cooking.csv', 'Proportion of population with primary reliance on clean fuels and technologies for cooking (%) - Residence area type: Total', 'Acesso_Combustivel_Limpo'),
    ('share-of-final-energy-consumption-from-renewable-sources.csv', '7.2.1 - Renewable energy share in the total final energy consumption (%) - EG_FEC_RNEW', 'Acesso_Energia_Renovavel')
]

for file_name, value_column, table_name in country_files:
    df = load_country_data(file_name, value_column)
    if df is not None:
        insert_paises(conn, df)
        insert_country_data(conn, df, table_name)
    else:
        print(f"Failed to load {file_name}.")

Inserted 215 countries
Inserted 6309 records into Acesso_Eletricidade
Inserted 2 countries
Inserted 6144 records into Acesso_Combustivel_Limpo
Inserted 16 countries
Inserted 4938 records into Acesso_Energia_Renovavel


### 5.3 Load and Insert Investment Data

In [ ]:
# Load and insert investment data
investment_file = 'international-finance-clean-energy.csv'
investment_column = '7.a.1 - International financial flows to developing countries in support of clean energy research and development and renewable energy production, including in hybrid systems (millions of constant 2021 United States dollars) - EG_IFF_RANDN - All renewables'

investment_data = load_country_data(investment_file, investment_column)
if investment_data is not None:
    insert_paises(conn, investment_data)
    insert_investment_data(conn, investment_data)
else:
    print("Failed to load investment data.")

Inserted 1 countries
Inserted 3519 investment records


### 5.4 Load and Insert Renewable Capacity Data

In [ ]:
# Load and insert renewable capacity data
renewable_file = 'renewable-electricity-generating-capacity-per-capita.csv'

try:
    capacity_df = pd.read_csv(os.path.join('./data', renewable_file))
    capacity_column = capacity_df.columns[3]  # Assuming the fourth column contains the data
    renewable_data = load_country_data(renewable_file, capacity_column)
    if renewable_data is not None:
        insert_paises(conn, renewable_data)
        insert_renewable_per_capita(conn, renewable_data)
    else:
        print("Failed to load renewable capacity data.")
except Exception as e:
    print(f"Error processing renewable capacity data: {e}")

Inserted 0 countries
Inserted 5083 renewable per capita records


### 5.5 Load and Insert HDI Data

In [ ]:
# Load and insert HDI data
hdi_data = load_hdi_data()
if hdi_data is not None:
    insert_paises(conn, hdi_data)
    insert_hdi_data(conn, hdi_data)
else:
    print("Failed to load HDI data.")

Trying to read HDI data with encoding: latin-1
Successfully loaded HDI data with encoding: latin-1
Inserted 11 countries
Inserted 6171 HDI records


## 6. Queries

### 6.1 Comparison of Average Electricity Access: Brazil vs. Global Average

This query compares Brazil's access to electricity with the global average over time. The medias CTE calculates the global average percentage of access to electricity per year, while the brasil CTE retrieves Brazil's yearly data. The final query joins these results by year, showing Brazil's percentage alongside the global average, ordered chronologically.

In [ ]:
query_1 = """
WITH media_mundial AS (
    SELECT 
        ano,
        AVG(porcentagem) AS media_global
    FROM 
        Acesso_Eletricidade
    GROUP BY 
        ano
),
brasil AS (
    SELECT 
        ae.ano,
        ae.porcentagem AS acesso_brasil
    FROM 
        Acesso_Eletricidade ae
    JOIN 
        Pais p ON ae.id_pais = p.id_pais
    WHERE 
        p.nome = 'Brazil'
)
SELECT 
    b.ano,
    b.acesso_brasil,
    m.media_global
FROM 
    brasil b
JOIN 
    media_mundial m ON b.ano = m.ano
ORDER BY 
    b.ano;

"""

### 6.2 Top 10 Countries with Renwable Energy

This query retrieves the top 10 countries with the highest renewable energy share in a specific year (here the example is with 2020), ranking them by percentage in descending order.

In [ ]:
query_2 = """
WITH ano_escolhido AS (
  SELECT 2020 AS ano  -- substitua por qualquer ano disponível
)
SELECT
  p.nome      AS pais,
  a.ano,
  a.porcentagem AS pct_renovavel
FROM
  Acesso_Energia_Renovavel AS a
  JOIN Pais AS p
    ON a.id_pais = p.id_pais
  JOIN ano_escolhido AS x
    ON a.ano = x.ano
ORDER BY
  a.porcentagem DESC
LIMIT 10;
"""

### 6.3 Correlation between HDI and Renewable Energy Generation per Capita

This query calculates the correlation between the Human Development Index (HDI) and renewable energy generation per capita for each year. It joins the IDH and Energia_Renovavel_Per_Capita tables on country and year, computes the correlation coefficient (CORR), and groups the results by year, displaying them in chronological order.

In [ ]:
query_3 = """
SELECT
  i.ano,
  CORR(i.indice, e.geracao_watts) AS correlacao_idh_geracao
FROM
  IDH AS i
  JOIN Energia_Renovavel_Per_Capita AS e
    ON i.id_pais = e.id_pais
   AND i.ano     = e.ano
GROUP BY
  i.ano
ORDER BY
  i.ano;
"""

### 6.4 Agents with Multiple Power Plants in Brazil

This query identifies agents who own more than one power plant. It counts the total number of plants (total_usinas) owned by each agent, groups the results by agent name, filters for agents with more than one plant, and orders the results in descending order of plant count.

In [ ]:
query_4 = """
SELECT
  ap.nome                AS agente,
  COUNT(u.id_usina)      AS total_usinas
FROM
  Agente_Proprietario AS ap
  JOIN Usina AS u
    ON ap.id_agente = u.id_agente_proprietario
GROUP BY
  ap.nome
HAVING
  COUNT(u.id_usina) > 1
ORDER BY
  total_usinas DESC;
"""

### 6.5 Power Plants by Fuel Type in Brazil

This query identifies the types of fuel used by power plants in Brazil and counts the number of distinct power plants (qtd_usinas) for each fuel type. It groups the results by fuel type, filters for power plants located in Brazil, and orders the results in descending order of the number of power plants.

In [ ]:
query_5 = """
SELECT
  ug.combustivel,
  COUNT(DISTINCT u.id_usina) AS qtd_usinas
FROM
  Unidade_Geradora ug
  JOIN Usina u ON ug.id_usina = u.id_usina
  JOIN Estado est ON u.id_estado = est.id_estado
  JOIN Subsistema_Estado se ON est.id_estado = se.id_estado
  JOIN Subsistema s ON se.id_subsistema = s.id_subsistema
  JOIN Pais p ON s.id_pais = p.id_pais
WHERE
  p.nome = 'Brazil'
GROUP BY
  ug.combustivel
ORDER BY
  qtd_usinas DESC;
"""

### 6.6 Total Power Generation Capacity by State in Brazil

This query calculates the total power generation capacity (capacidade_total_mw) for each state in Brazil. It aggregates the effective power (potencia_efetiva) of all generating units, groups the results by state, and orders them in descending order of total capacity. 

In [ ]:
query_6 = """
SELECT
  est.nome AS estado,
  SUM(ug.potencia_efetiva) AS capacidade_total_mw
FROM
  Unidade_Geradora ug
  JOIN Usina u ON ug.id_usina = u.id_usina
  JOIN Estado est ON u.id_estado = est.id_estado
  JOIN Subsistema_Estado se ON est.id_estado = se.id_estado
  JOIN Subsistema s ON se.id_subsistema = s.id_subsistema
  JOIN Pais p ON s.id_pais = p.id_pais
WHERE
  p.nome = 'Brazil'
GROUP BY
  est.nome
ORDER BY
  capacidade_total_mw DESC;
"""

### 6.7 Percentage of renewable vs. non-renewable power plants by state

This query calculates the percentage of renewable energy power plants in each state of Brazil. It first determines the total number of power plants (total_usinas) and the number of renewable energy power plants (usinas_renovaveis) for each state. Renewable energy sources include biomass, solar, wind, and hydroelectric. The final result includes the state name, total power plants, renewable power plants, and the percentage of renewable power plants, ordered by the percentage in descending order.

In [ ]:
query_7 = """
WITH tot_estado AS (
  SELECT
    est.id_estado,
    est.nome AS estado,
    COUNT(u.id_usina) AS total_usinas
  FROM Usina u
    JOIN Estado est ON u.id_estado = est.id_estado
    JOIN Subsistema_Estado se ON est.id_estado = se.id_estado
    JOIN Subsistema s ON se.id_subsistema = s.id_subsistema
    JOIN Pais p ON s.id_pais = p.id_pais
  WHERE p.nome = 'Brazil'
  GROUP BY est.id_estado, est.nome
), ren_estado AS (
  SELECT
    est.id_estado,
    COUNT(DISTINCT u.id_usina) AS usinas_renovaveis
  FROM Unidade_Geradora ug
    JOIN Usina u ON ug.id_usina = u.id_usina
    JOIN Estado est ON u.id_estado = est.id_estado
    JOIN Subsistema_Estado se ON est.id_estado = se.id_estado
    JOIN Subsistema s ON se.id_subsistema = s.id_subsistema
    JOIN Pais p ON s.id_pais = p.id_pais
  WHERE
    p.nome = 'Brazil'
    AND ug.combustivel IN ('BIOMASSA', 'SOLAR', 'EÓLICA', 'HÍDRICA')  -- exemplo de renováveis
  GROUP BY est.id_estado
)
SELECT
  t.estado,
  t.total_usinas,
  COALESCE(r.usinas_renovaveis, 0) AS usinas_renovaveis,
  ROUND(
    COALESCE(r.usinas_renovaveis, 0)::decimal
    / t.total_usinas * 100
  , 2) AS perc_renovaveis
FROM
  tot_estado t
  LEFT JOIN ren_estado r ON t.id_estado = r.id_estado
ORDER BY
  perc_renovaveis DESC;
"""

## 7 Function to Run and Save Queries

In [ ]:
# Create output directory for query results
OUTPUT_DIR = './query_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

queries = {
    'query_1': query_1,
    'query_2': query_2,
    'query_3': query_3,
    'query_4': query_4,
    'query_5': query_5,
    'query_6': query_6,
    'query_7': query_7
}

# Function to run queries and save results to CSV
def run_queries_and_save(queries):
    try:
        # Connect to the database
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        for query_name, query in queries.items():
            print(f"Running {query_name}...")
            cursor.execute(query)
            # Fetch all results
            columns = [desc[0] for desc in cursor.description]
            rows = cursor.fetchall()
            # Convert to DataFrame
            df = pd.DataFrame(rows, columns=columns)
            # Save to CSV
            output_path = os.path.join(OUTPUT_DIR, f"{query_name}.csv")
            df.to_csv(output_path, index=False)
            print(f"Saved {query_name} results to {output_path}")

    except Exception as e:
        print(f"Error: {e}")

## 8 Run and Save Queries

In [ ]:
run_queries_and_save(queries)

Running query_1...
Saved query_1 results to ./query_results/query_1.csv
Running query_2...
Saved query_2 results to ./query_results/query_2.csv
Running query_3...
Saved query_3 results to ./query_results/query_3.csv
Running query_4...
Saved query_4 results to ./query_results/query_4.csv
Running query_5...
Saved query_5 results to ./query_results/query_5.csv
Running query_6...
Saved query_6 results to ./query_results/query_6.csv
Running query_7...
Saved query_7 results to ./query_results/query_7.csv


## 9 Close Database Connection

In [ ]:
# Close the database connection
if cursor:
    cursor.close()
if conn:
    conn.close()
print("Database connection closed.")

Database connection closed.
